<center><h2> Data collection </h2></center>

For the data collection part we've scraped <a href='https://www.magicbricks.com/'>MagicBricks</a> which is a platform for property buyers & sellers.<br>
### Tech stack used:
* Python
* Beautiful Soup
* Selenium
* Pandas

In [10]:
# import required libraries
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep
import pandas as pd
from datetime import datetime

# path where required driver file is stored
path = 'C:\\Users\\anike\\PycharmProjects\\Web Scraping\\chromedriver.exe'

# OOPS implementation
class MagicBricksScraper:

    def __init__(self):
        i = 0,
        c = 'Pune'
        #self.driver_path = ''
        self.driver = ''
        self.final_data = []
        self.URL = "https://www.magicbricks.com/property-for-sale/residential-real-estate?proptype=Multistorey-Apartment," \
                   "Builder-Floor-Apartment,Penthouse,Studio-Apartment,Residential-House,Villa&page=" + str(
            i) + "&cityName=" + str(c)

    def load(self, path):
        """Loads the website"""
        
        self.driver = webdriver.Chrome(executable_path=path)
        self.driver.get('https://www.magicbricks.com')
        self.driver.maximize_window()

    def extract(self, cities, pages):
        """Extracts all cards available on the current loaded page

        Arguments:
            cities -- list of cities for which to extract data
            pages -- number of pages to search for each city

        Returns:
            final_data -- list of dictionaries containing extracted data
        """

        for c in cities:
            
            for i in range(pages):
                
                self.driver.get(
                    "https://www.magicbricks.com/property-for-sale/residential-real-estate?proptype=Multistorey-Apartment,Builder-Floor-Apartment,Penthouse,Studio-Apartment,Residential-House,Villa&page="+str(i)+"&cityName="+str(c))
                print(f'City: {c}, page: {i}')
                
                # extract page source
                soup = BeautifulSoup(self.driver.page_source, 'html.parser')
                
                # find all div tags inside source
                div_tags = soup.find_all('div', 'mb-srp__list')
                
                self.get_data(div_tags, c)

        return self.final_data

    def get_data(self, div_tags, city):
        """Extracts all data from a card"""

        for d in div_tags:
            # empty dictionary to store a record
            data_dic = {}

            try:
                # title of the card
                title = d.find_all('h2', 'mb-srp__card--title')[0].text

                # society name
                society_name = d.find_all('a', 'mb-srp__card__society--name')[0].text

                # price of property
                price = d.find_all('div', 'mb-srp__card__price--amount')[0].text

            except IndexError:
                return

            # label and values
            label = d.find_all('div', 'mb-srp__card__summary--label')
            value = d.find_all('div', 'mb-srp__card__summary--value')

            prop_id = d.get('id')

            # metadata
            meta = d.find_all('meta')

            name = ''
            description = ''
            url = ''
            address = ''
            longitude = ''
            latitude = ''
            number_of_rooms = ''

            for m in meta:
                if m['itemprop'] == 'name':
                    name = m['content']
                if m['itemprop'] == 'description':
                    description = m['content']
                if m['itemprop'] == 'url':
                    url = m['content']
                if m['itemprop'] == 'addressLocality':
                    address = m['content']
                if m['itemprop'] == 'longitude':
                    longitude = m['content']
                if m['itemprop'] == 'latitude':
                    latitude = m['content']
                if m['itemprop'] == 'number_of_rooms':
                    number_of_rooms = m['content']

            data_dic = {'Property Id': prop_id,
                        'Title': title,
                        'Society Name': society_name,
                        'Price': price,
                        'Name': name,
                        'Description': description,
                        'URL': url,
                        'Address': address,
                        'City': city,
                        'Latitude': latitude,
                        'Longitude': longitude,
                        'Number of Rooms': number_of_rooms}

            # creating key-value pair using label and values
            for i in range(len(label)):
                data_dic[label[i].text] = value[i].text
                

            self.final_data.append(data_dic)

In [11]:
import pandas as pd
from datetime import datetime
import warnings

warnings.filterwarnings(action='ignore')

cities = ['Gurgaon', 'Noida', 'Ghaziabad', 'Greater-Noida', 'Bangalore', 'Mumbai', 'Pune', 'Hyderabad', 'Kolkata',
          'Chennai', 'New-Delhi', 'Ahmedabad', 'Navi-Mumbai', 'Thane', 'Faridabad', 'Bhubaneswar', 'Bokaro-Steel-City',
          'Vijayawada', 'Vrindavan', 'Bhopal', 'Gorakhpur', 'Jamshedpur', 'Agra', 'Allahabad', 'Jodhpur', 'Aurangabad',
          'Jaipur', 'Mangalore', 'Nagpur', 'Guntur', 'Navsari', 'Palghar', 'Salem', 'Haridwar', 'Durgapur', 'Madurai',
          'Manipal', 'Patna', 'Ranchi', 'Raipur', 'Sonipat', 'Kottayam', 'Kozhikode', 'Thrissur', 'Tirupati', 'Trivandrum',
          'Trichy', 'Udaipur', 'Vapi', 'Varanasi', 'Vadodara', 'Visakhapatnam', 'Surat', 'Kanpur', 'Kochi', 'Mysore', 'Goa',
          'Bhiwandi', 'Lucknow', 'Nashik', 'Guwahati', 'Chandigarh', 'Indore', 'Coimbatore', 'Dehradun', 'Latur', 'Gandhinagar',
          'Meerut', 'Warangal', 'Bikaner']

path = 'chromedriver.exe'
obj = MagicBricksScraper()
obj.load(path)
final_data = obj.extract(['Pune'], 2)  # pass the cities list and no. of pages
print(len(final_data))
# print(final_data)

ext1 = '.xlsx'
ext2 = '.csv'
name = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
path1 = 'MagicBricks-' + name + ext1
# path2 = 'MagicBricks-' + name + ext2

df = pd.DataFrame(final_data)
df.to_excel(path1, index=False)
# df.to_csv(path2, index=False)
obj.driver.quit()

City: Pune, page: 0
City: Pune, page: 1
60
